# 透過 Python 擷取網頁上的資料

[Occam's Razor](https://zh.wikipedia.org/wiki/%E5%A5%A5%E5%8D%A1%E5%A7%86%E5%89%83%E5%88%80)

在大數據變成顯學的時代，今天我們要做資料分析，若**沒有大量的資料，是無法分析出任何有價值的資訊出來的。**

但是在一般情況下，我們都沒有大量的資料可以做分析，但幸運的是，由於網路的蓬勃發展，獲取任何一個領域的資料比起過往相對就變得容易許多。但是，由於網頁上的資料非常多，若透過手動的方式截取資料，就不是一個很有效率的方法了。

因此這時候，從網路上蒐集資料這個議題也變得更加重要，只需一個簡單的程式，就能透過低成本並且自動化的方式從網頁上獲得大量的資料，因此**學習與實作網頁爬蟲成為一個投資報酬率極高的事務**，而 Python 語言由於生態系龐大，套件衆多，也讓用 Python 實作爬蟲比起其他語言相對簡單許多。

我們這節課要做的事，就是實作出一個台股股價爬蟲，到 [Yahoo 奇摩](https://tw.stock.yahoo.com/q/q?s=2330)上截取資料：

![](https://drive.google.com/uc?export=download&id=1oEKgI1DGTx9nfJxW9GpfvUalSgfFpypl)

最後再將截取到的股價資料輸出至 Excel 工作表：

![](https://drive.google.com/uc?export=download&id=1PrcTFvmMiepoX4Gk_caWKJjhhQnlXFzI)

## 網絡基礎： HTTP 溝通協定

HTTP 用白話講，就像是**電腦與電腦之間的共同語言**，電腦需要通過這個共同語言，才能在網絡上面互相溝通。

![](https://drive.google.com/uc?export=download&id=1UiDUXriZVo83ePdiSQTLS4QWnCN_YsVu)

而一個 HTTP 的請求 (Request) 是根據 **HTTP 動詞**(HTTP Verb) 以及**網址**才能運作，舉例來説，當你在輸入 https://www.google.com/ ，連至 Google 的首頁時，整個背後的運作流程是：

1. 用戶端 (你的瀏覽器) 針對 Google 的雲端伺服器發送了一個 GET Request
2. 而 Google 的雲端伺服器在收到請求後，將需要呈現該網頁的資料都計算完成
3. 接著 Google 的雲端伺服器會回傳一個回應 (Response)，這個 Response 内通常就包含了一個 html 檔案
4. 用戶端 (你的瀏覽器) 在下載了請求回傳的 html 檔案之後，將 html 程式碼渲染成網頁，呈現給使用者

## HTTP Request 的種類

一般來説，Request 有 GET 與 POST：

- GET 代表我需要查詢 / 顯示資料，像是 GET https://www.facebook.com/ 代表查詢 facebook 首頁
- POST 代表我需要新增資料，通常用於網頁上的表單 POST https://www.foodpanda.com/orders 代表新增訂單

## HTTP response 的種類

- 一般伺服器通常是回傳 html 網頁檔案
- 但若是一些功能像是下載 / 輸出報表，Response 則是一個 xlsx / csv 檔案


## 從 Excel 的角度來理解...

像是 FB, Google, Yahoo 等網站，**其實都像是一個個運行在雲端上的 Excel 函數**。

而要使用該函數時，就必須透過一個 HTTP 的請求 (Request) 來呼叫該函數。讓瀏覽器發送 HTTP Request 的方法就是輸入網址，就像是在 Excel 上輸入公式一樣。

而該公式若執行成功，不同於 Excel 是將結果顯示在工作表上，HTTP Request 的結果就是一個 Response，該 Response 通常是顯示在瀏覽器上的網頁，也有可能是 xlsx / csv 檔案


## 用 Python 實作爬蟲

Python 用來實作爬蟲的兩個主流套件：

- BeautifulSoup [官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

- PyQuery [官方文件](https://pythonhosted.org/pyquery/)

簡單來説：

**BeautifulSoup** 是比較貼近**程式設計師**的角度去思考

**PyQuery** 貼近**網頁開發者**的角度去思考

在這堂課考量到並不是每一個人都具備網頁開發的背景，因此我們會使用 BeautifulSoup 套件來實作爬蟲

## 範例網頁：

http://pythonscraping.com/pages/warandpeace.html

*以上練習用網頁由 Ryan Mitchell 維護，可以的話，大家可以買一本書支持他：[連結](https://www.books.com.tw/products/0010800965)

## 網頁開發 101

任何網頁都是由 **html 標籤(tag)** 所組成，基本結構如下
```html
<標籤名稱 class="類別名稱">內容</標籤名稱>
<標籤名稱 id="id名稱">內容</標籤名稱>
```
今天我們要擷取的任何內容，一定是被包裹在在某一個標籤裡面
而今天若網頁開發者需要改變任何一個標籤的**樣式**，就需要用到 **css** 語法
以上面的網頁為例，人名都是以綠色顯示，所以就先宣告一個名為 **green** 的 css 類別:

```html
<style>
.green{
	color:#55ff55;
}
</style>
```

若今天希望讓一個標籤的内容文字變成綠色，可以使用定義好的 .green 這個 css 類別：

```html
<span class="green">Prince Vasili Kuragin</span>
```

*想了解更多 html 可以看一下 Mozilla 官網的教學：[HTML 基礎](https://developer.mozilla.org/zh-TW/docs/Learn/Getting_started_with_the_web/HTML_basics)

## 來實作吧...

參考:[Requests Documentation](https://requests.readthedocs.io/en/master/user/quickstart/)

今天我們要截取網頁内一些特定的内容，我們必須先將網頁下載下來做分析：

```python
import requests

# 針對網頁發送 GET Reqeust
res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
# 將回傳的 Response 内的文字顯示出來
res.text
#'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t...
```

可以看到我們的範例網頁的 html 程式碼是被濃縮成了一個龐大的字串，而我們想要截取的資料一定是在這個字串内的某個地方，接下來就要思考如何能夠在這個字串過濾出我們有興趣的資訊

In [6]:
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html") # <Response [200]>
res.text

'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy \'faithful slave,\' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.</span>"\n<p/>\nIt was in July, 1805, and the speaker was the well-known <span class="green">Anna\nPavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya\nFedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man\nof high rank

## 使用 標籤 「類別名稱」取得資料

要擷取資料前，首先需要透過方法**選擇**到該標籤
##  BeautifulSoup 套件

典故來自 Alice in WonderLand 裏面一首同名的詩，由假海龜 (Mock Turtle) 所唱，影射英國料理假海龜湯...

有興趣自己可以去 Google, 不多説了...

簡單來說，BeautifulSoup 就是 Python 與 html 網頁的翻譯機，它會把我們方才截取的網頁字串解析、並且放入一個類似 html 格式的資料下，方便 Python 的使用者能夠截取透過結構化的方式截取到需要的資料。

```python
from bs4 import BeautifulSoup
import requests

# 針對網頁發送 GET Reqeust
res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
# 將回傳的 Response 内的文字用 BeautifulSoup 解析
html = BeautifulSoup(res.text, "html.parser")
html
```

In [20]:
from bs4 import BeautifulSoup
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
html = BeautifulSoup(res.text, "html.parser")
html

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

接下來我們希望將放置人物名稱的標籤，也就是 **span** 標籤，搜尋出來：

```python
html.findAll("span", { "class": "green"})
```

但是我們馬上發現到，範例網頁内的除了人名外，還有人物的對白也是被放在 `span` 標籤内，我們截取到了很多不需要的東西，所以希望能夠增加一些過濾的條件，我們注意到了所有人物標籤的 `class` 屬性皆爲 **green**，所以我們就修改我們原本的程式碼：

```python
name_list = html.findAll("span", { "class": "green"})
print(name_list)
#[<span class="green">Anna
# Pavlovna Scherer</span>, <span class="green">Empress Marya
# Fedorovna</span>, <span class="green">Prince Vasili Kuragin</span>, <span class="green">Anna # Pavlovna</span>, <span class="green">St. Petersburg</span>,
```

讓 BeautifulSoup 過濾出所有 `class` 為 **green** 的 `span` 標籤

In [29]:
name_list = html.findAll("span", { "class": "green"})
for n in name_list:
    print(n.getText())
    #.text .string .getText()

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


# 截取出標籤内的文字

現在我們截取到的，是一個 BeautifulSoup 的 html 標籤物件的集合

一個 html 標籤物件其實是個複雜的存在，如同包含很多不同的屬性

現在我們希望能夠將每一個 html 標籤中的文字提取出來：

```python
# 提取出第一個 html 標籤物件
tag = name_list[0]
# 截取該標籤内的文字
tag.text
```

確認以上方法可以提取到資料之後，我們就來寫個 for loop 將每一個 html 標籤的文字印出來：

```python
for tag in name_list:
    print(tag.text)
```

## 另外再介紹一下 find() 方法...

```python
from bs4 import BeautifulSoup
import requests

file = requests.get("http://pythonscraping.com/pages/warandpeace.html")
html = BeautifulSoup(file.text, 'html.parser')
# 我們把該網頁使用 'green' css 類別的 span 標簽過濾出來...
name = html.find("span", {"class": "green"})
# 但是 find 只會回傳一筆資料
print(name)
# <span class="green">Anna Pavlovna Scherer</span>
```

In [15]:
from bs4 import BeautifulSoup
import requests

file = requests.get("http://pythonscraping.com/pages/warandpeace.html")
html = BeautifulSoup(file.text, 'html.parser')
# 我們把該網頁使用 'green' css 類別的 span 標簽過濾出來...
name = html.find("span", {"class": "green"})
# 但是 find 只會回傳一筆資料
print(name)
# <span class="green">Anna Pavlovna Scherer</span>

<span class="green">Anna
Pavlovna Scherer</span>


## findAll() vs find()

`findAll()` 是找出**所有**符合條件的標簽

`find()` 是找到**第一筆**符合條件的資料

`select()` Always use css selectors when chaining tags or using `tag.classname`. [Beautifulsoup : Is there a difference between .find() and .select()](https://stackoverflow.com/questions/38028384/beautifulsoup-is-there-a-difference-between-find-and-select-python-3-x)

# 網頁爬蟲實戰：(臺股爬蟲)

我們想要截取資料的網頁：[Yahoo Stock 台積電](https://tw.stock.yahoo.com/q/q?s=2330)

![](https://drive.google.com/uc?export=download&id=1samMUBW19ooC7UsntphFGG6QKjL5337R)

網址是：

https://tw.stock.yahoo.com/q/q?s=2330

```python
# 來試試看爬 yahoo stock 的網頁...
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
# 用 html 格式解碼 爬下來的檔案
html = BeautifulSoup(res.text, 'html.parser')
print(html)
```


In [16]:
# 來試試看爬 yahoo stock 的網頁...
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
# 用 html 格式解碼 爬下來的檔案
html = BeautifulSoup(res.text, 'html.parser')
print(html)

<!DOCTYPE doctype html public "-//w3c//dtd html 4.01//en" "http://www.w3.org/tr/html4/strict.dtd">

<html>
<head>
<meta content="false" name="oath:guce:product-eu"/>
<meta content="guce.yahoo.com" name="oath:guce:consent-host"/>
<meta content="zh-Hant-TW" name="oath:guce:locale"/>
<meta content="false" name="oath:guce:report-only"/>
<meta content="true" name="oath:guce:inline-consent"/>
<script async="" src="https://s.yimg.com/oa/guce.js"></script>
<!-- Rapid begin. -->
<script language="JavaScript" src="https://s.yimg.com/ss/rapid3.js"></script>
<script language="JavaScript" src="/__rapid-worker-1.2.js"></script> <!-- rapid should require worker file itself, this should be unnessary, but require it anyway. -->
<script language="JavaScript">
    YAHOO.i13n.WEBWORKER_FILE = "/__rapid-worker-1.2.js";
    var pageParams = { site: 'finance', lang: 'zh-Hant-TW', mrkt: 'tw' };
    if (typeof articlePageParams === 'object') {
        pageParams = articlePageParams;
    }
    var stockSiteRapi

# 問題是找到有用的資料如同大海撈針...

---
## 分析一下我們要爬的網頁

收盤價是被封裝在一個 **table** 標籤内部的一個 **td** 標籤


---
## table 標籤

一般網頁若要呈現重要的資訊，都會將資訊以網頁表格的形式呈現

今天若要利用 **html** 實作網頁上的表格，需要名爲 **table** 的標籤

換句話説，今天網頁上我們有興趣的資料，十之八九都是被封裝在 **table** 底下

把 **table** 的結構搞懂，就成了一件重要的事情。

---
# html table 標簽的結構

網頁上的資料大多都是匯整在表格、而 html 的表格則是由 table 標簽構成的：

![](https://drive.google.com/uc?export=download&id=1dmYx5qOD21tPWHATzIm6IxDnfO96sUac)

該 **table** 標籤内有兩個 **tr** 標簽

第二個 tr 標籤内的第八個 td 標簽是我們要的收盤價

![](https://drive.google.com/uc?export=download&id=109M71HQtR-kfZMklVit7AZRpIH4zhfe_)

---
# html 標簽的關聯

簡單來説，就是一個樹狀圖的概念：

![](https://drive.google.com/uc?export=download&id=1n2yZlQtj7EK2_NNIBXvEVGeEKBogfgYZ)

---
# 延申閲讀

HTML Table 教學

w3 school： [連結](https://www.w3schools.com/html/html_tables.asp)

Mozilla：[連結](https://developer.mozilla.org/zh-TW/docs/Web/HTML/Element/table)

---

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤：
html.findAll(text='個股資料')[0]
# '個股資料'
```

In [31]:
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤：
html.findAll(text='個股資料')[0]

'個股資料'

## 用 .parent 關聯到上一層

```python
html.findAll(text='個股資料')[0].parent
# <th align="center">個股資料</th>
```

In [37]:
html.findAll(text='個股資料')[0].parent

<th align="center">個股資料</th>

## 接下來關聯到 table 層

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
print(table)
```

In [33]:
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
print(table)

<table border="2" width="750">
<tr bgcolor="#fff0c1">
<th align="center">股票<br/>代號</th>
<th align="center" width="55">時間</th>
<th align="center" width="55">成交</th>
<th align="center" width="55">買進</th>
<th align="center" width="55">賣出</th>
<th align="center" width="55">漲跌</th>
<th align="center" width="55">張數</th>
<th align="center" width="55">昨收</th>
<th align="center" width="55">開盤</th>
<th align="center" width="55">最高</th>
<th align="center" width="55">最低</th>
<th align="center">個股資料</th>
</tr>
<tr>
<td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>
<td align="center" bgcolor="#FFFfff" nowrap="">10:48</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><b>328.5</b></td>
<td align="center" bgcolor="#FFFfff" nowrap="">328.0</td>
<td align="center" bgcolor="#FFFfff" nowrap="">328.5</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><font color="#ff0000">△3.5
                <td align="ce

## 找尋 table 裡第二個 tr 標籤內所有的 td 標籤

陣列裡面12格依序代表：

股票 代號 時間 成交	買進 賣出 漲跌 張數 昨收 開盤 最高 最低 個股資料

```python
data_row = table.select('tr')[1].select('td')
data_row
```

In [51]:
# len = 12 (股票	代號	時間	成交	買進	賣出	漲跌	張數	昨收	開盤	最高	最低	個股資料)
data_row = table.select('tr')[1].select('td')
data_row 

[<td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">11:39</td>,
 <td align="center" bgcolor="#FFFfff" nowrap=""><b>328.5</b></td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">328.0</td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">328.5</td>,
 <td align="center" bgcolor="#FFFfff" nowrap=""><font color="#ff0000">△3.5
                 <td align="center" bgcolor="#FFFfff" nowrap="">21,553</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">325.0</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">329.0</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">329.5</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">324.5</td>
 <td align="center" class="tt" width="137">
 <a href="/q/ts?s=2330">成交明細</a><br/><a href="/q/ta?s=2330">技術</a>　<a href="/q/h?s=2330">新聞</a><a href="/d/s/company_2330.html"><br/>基本</a>　<a href="/d/s/credit_2330.html

## 選取該 row 第八個 td 標籤，擷取標籤內文字
```python
last_close = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")
```

In [38]:
last_close = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")

台積電昨日收盤價：$325.0


## 完成版網頁爬蟲


```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.select('tr')[1].select('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")
```

In [39]:
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.select('tr')[1].select('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")

台積電昨日收盤價：$325.0


## 另一種解法...

![](https://drive.google.com/uc?export=download&id=1IJp5dze4stcwni0FNs2ThIjzYM_vYp4o)

注意 **tt** 是一個獨特的 css class，而 **tt** 只有被套用在最後一個 **td** 標籤上

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 尋找 class 屬性為 tt 的 td 標籤
last_td = html.find("td", {"class": "tt"})
# find_previous_sibling('td') 代表尋找前一個 (左邊) td 標籤
last_close = last_td.find_previous_sibling("td").find_previous_sibling("td").find_previous_sibling("td").find_previous_sibling("td").text

print(f"台積電昨日收盤價：${last_close}")
```

## 隨堂練習

請試試看從 Yahoo 股市網頁將 2330 的開盤價、最高價、與最低價讀取出來：

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
table = html.findAll(text='個股資料')[0].parent.parent.parent
data_row = table.select('tr')[1].select('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = data_row[7].text
open_price = ________________
high_price = ________________
low_price = ________________
close_price = ________________
```

In [44]:
# 目標 > Get 開高低收
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
table = html.findAll(text='個股資料')[0].parent.parent.parent
data_row = table.select('tr')[1].select('td')

# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close  = data_row[7].text # 昨收
open_price  = data_row[8].text # 開
high_price  = data_row[9].text # 高
low_price   = data_row[10].text # 低
close_price = data_row[2].text # 現在的收盤價 (今日的下午1:30最後一筆為今日收盤價)

data = {
    "開盤價": open_price, 
    "最高價": high_price,
    "最低價": low_price,
    "成交價": close_price,
    "昨日收盤價": last_close
}
data

{'開盤價': '329.0',
 '最高價': '329.5',
 '最低價': '324.5',
 '成交價': '328.5',
 '昨日收盤價': '325.0'}

## 將截取到的股價資訊寫入 Excel

用 Python 打造網頁爬蟲十分簡單，但是資料是需要長期纍積才能從中分析出有價值的資訊，因此，我們勢必需要一個媒介將大量的歷史資料存取下來，幸運的是，Excel 在這樣的場景下，已經足夠使用，接下來我們就來整合我們之前學會的技術，將網頁爬蟲截取下來的資料寫入 Excel 内：

![](https://drive.google.com/uc?export=download&id=1bmXQgdMhj3L7NYvonKjyLSdsUmZPlqGf)


用 xlwings 開啓 **tw_stock_portfolio.xlsx**：

```python
import xlwings as xw
from xlwings.constants import Direction
import time

wb = xw.Book(r"TW2330.xlsx")
sheet = wb.sheets["TW2330"]
```

偵測最後一個 row:

```python
last_row = sheet.range("A1").end("down").row
```

In [48]:
import xlwings as xw
import time

wb = xw.Book(r"TW2330.xlsx")
sheet = wb.sheets["TW2330"]

last_row = sheet.range("A1").end("down").row
last_row

25

## 產生格式化的時間字串

```python
import time

time.strftime("%Y/%m/%d")
```

## 將時間與昨日收盤價寫入 Excel 

```python
sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"F{last_row+1}").value = last_close
```

In [50]:
import time
time.strftime("%Y/%m/%d")

sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = open_price
sheet.range(f"C{last_row+1}").value = high_price
sheet.range(f"D{last_row+1}").value = low_price
sheet.range(f"E{last_row+1}").value = close_price
sheet.range(f"F{last_row+1}").value = last_close

# 完成版程式碼

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import xlwings as xw
from xlwings.constants import Direction

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.select('tr')[1].select('td')
# 截取臺積電當日的交易價格(OHLC)，以及昨日收盤價
last_close = data_row[7].text
open_price = data_row[8].text
high_price = data_row[9].text
low_price = data_row[10].text
close_price = data_row[2].text

print(f"台積電今日收盤價：${close_price}")

wb = xw.Book(r"檔案 TW2330.xlsx 的路徑")
date = time.strftime("%Y/%m/%d")
sheet = wb.sheets["TW2330"]

last_row = sheet.range("A1").end("down").row

sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = open_price
sheet.range(f"C{last_row+1}").value = high_price
sheet.range(f"D{last_row+1}").value = low_price
sheet.range(f"E{last_row+1}").value = close_price
sheet.range(f"F{last_row+1}").value = last_close

# 今天若想要把整個投資組合的資料截取到 Excel 内

我們發現到，在 Yahoo 奇摩股市的網頁上，不同股票的資料**都會顯示在 html 結構一樣的網頁上**，而唯一不同的只有**網址**，所以我們其實可以把同樣的程式碼封裝到一個函數内：

```python
from bs4 import BeautifulSoup
import requests
import time
import xlwings as xw

# 輸入股票代號，回傳該股票的收盤價
def yahoo_stock_crawler(stock_id):
    doc = requests.get(f"https://tw.stock.yahoo.com/q/q?s={stock_id}")
    html = BeautifulSoup(doc.text, 'html.parser')
    # 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
    table = html.findAll(text="個股資料")[0].parent.parent.parent
    # 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
    data_row = table.select("tr")[1].select("td")

    # 回傳一個字典
    return {
        "open": data_row[8].text,
        "high": data_row[9].text,
        "low": data_row[10].text,
        "close": data_row[2].text,
        "lastClose": data_row[7].text
    }
```

最後，我們就可以將一整個投資組合的資料截取下來，並存入相對應的工作表内

```python
wb = xw.Book(r"tw_stock_portfolio.xlsx")
date = time.strftime("%Y/%m/%d")
# 定義所有投資組合的股票代號
stocks = [2884, 2330, 2317]

for stock in stocks:
    # 截取到該股票代號的收盤價
    data = yahoo_stock_crawler(stock)
    # 截取相對應的工作表
    sheet = wb.sheets[f"TW{stock}"]
    # 偵測該工作表的最後一行行數
    last_row = sheet.range("B1").end("down").row
    # 將日期寫入 A 欄
    sheet.range(f"A{last_row+1}").value = date
    sheet.range(f"B{last_row+1}").value = data["open"]
    sheet.range(f"C{last_row+1}").value = data["high"]
    sheet.range(f"D{last_row+1}").value = data["low"]
    sheet.range(f"E{last_row+1}").value = data["close"]
    sheet.range(f"F{last_row+1}").value = data["lastClose"]
```

# 小結：

1. 學習與實作網頁爬蟲是一個**投資報酬率極高的事務**
2. Python 語言由於使用者衆多，**與爬蟲相關的套件、解決方案、與教學也多，讓實作變得相對簡單**
3. 實作上，最困難的部分在於**解析網頁的 html 結構**
4. 網頁的資料很大的機率都是被封裝在 **table** 這個 html 標簽下
5. 但若今天**網頁改版，原先寫好的爬蟲就有可能截取不到資料**

## 參考文獻

- 想查看 BeautifulSoup 套件的功能請看一下官方的中文文件：[BeautifulSoup 官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

- 想瞭解更多用 Python 實作網頁爬蟲的技術，可以參考這本書：[網站截取使用 Python](https://www.books.com.tw/products/0010800965?gclid=Cj0KCQjwr-_tBRCMARIsAN413WSYNHsbFOht8DIEqJ5sJZMzvoX5C8i_RlPo3df7KIW2mf-LQovtZzUaAmIcEALw_wcB)

# 功課：匯率爬蟲

請寫一個網頁爬蟲，截取臺灣銀行牌告匯率網頁：

http://rate.bot.com.tw/xrt?Lang=zh-TW


並且將資料用以下格式呈現在 Excel 内：
![](https://drive.google.com/uc?export=download&id=1YCl-QcAJCW951AhosB3HhuV7Fwy3hjMZ)